# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

An equal-weight portfolio is an investment strategy where each asset in the portfolio is allocated the same percentage of the total investment, regardless of the asset's market value, size, or other characteristics. This approach contrasts with market-capitalization-weighted portfolios, where larger companies or assets have a proportionally greater influence on the portfolio's performanc


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from tokenz import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol='aapl'
url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
outer_data = requests.get(url).json()
outer_data

[{'beta': 1.0391476142440639,
  'totalCash': 67150000000,
  'currentDebt': 123822000000,
  'revenue': 381623000000,
  'grossProfit': 173966000000,
  'totalRevenue': 381623000000,
  'EBITDA': 129629000000,
  'revenuePerShare': 24.89,
  'revenuePerEmployee': 2596074.83,
  'debtToEquity': 4.54768579669515,
  'profitMargin': 0.2630580441954494,
  'enterpriseValue': 2893872794960,
  'enterpriseValueToRevenue': 7.58,
  'priceToSales': 7.48,
  'priceToBook': 38.5,
  'forwardPERatio': 28.24659325127216,
  'pegRatio': 3.0530400663553263,
  'peHigh': 11.867282676107855,
  'peLow': 7.976472620192518,
  'week52highDate': '2023-12-14',
  'week52lowDate': '2024-04-19',
  'putCallRatio': None,
  'companyName': 'Apple Inc',
  'marketcap': 2856432794960,
  'week52high': 198.57,
  'week52low': 163.63,
  'week52highSplitAdjustOnly': 199.62,
  'week52highDateSplitAdjustOnly': '2023-12-14',
  'week52lowSplitAdjustOnly': 164.08,
  'week52lowDateSplitAdjustOnly': '2024-04-19',
  'week52change': 0.09411002536

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data = outer_data[0]
data['year1ChangePercent']

0.09094018335385036

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [5]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker'][:100]:
    # print(symbol)
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol}?token={IEX_CLOUD_API_TOKEN}'
    outer_data = requests.get(batch_api_call_url).json()
    dataOne = outer_data[0]
    dataTwo = outer_data[1]
    # print(dataOne['latestPrice'])
    # print(dataTwo['year1ChangePercent'])
    new_row = pd.Series([symbol, dataOne['latestPrice'], dataTwo['year1ChangePercent'],'N/A'], index = my_columns)
    final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)

C:\Users\Aryan\AppData\Local\Temp\ipykernel_27812\2952729528.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)


In [6]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,151.28,0.167409,N/A
1,AAL,15.19,0.064907,N/A
2,AAP,75.64,-0.377378,N/A
3,AAPL,187.43,0.090940,N/A
4,ABBV,161.59,0.140514,N/A
...,...,...,...,...
95,CINF,118.21,0.201585,N/A
96,CL,94.57,0.180952,N/A
97,CLX,138.55,-0.135788,N/A
98,CMA,54.00,0.766446,N/A


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [33]:
for symbol in stocks['Ticker']:
    # print(symbol_strings)
    # batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol}?token={IEX_CLOUD_API_TOKEN}'
    outer_data = requests.get(batch_api_call_url).json()
    dataOne = outer_data[0]
    dataTwo = outer_data[1]
    print(dataOne['latestPrice'])
    print(dataTwo['year1ChangePercent'])

151.28
0.1674089383894719


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:100]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,ANET,313.66,1.235230,N/A
1,AVGO,1380.03,1.191866,N/A
2,AMAT,209.82,0.806061,N/A
3,CMA,54.00,0.766446,N/A
4,CAT,358.18,0.733737,N/A
...,...,...,...,...
95,AAP,75.64,-0.377378,N/A
96,ABMD,381.02,NaN,N/A
97,ALXN,182.50,NaN,N/A
98,ATVI,94.42,NaN,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

6900


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,ANET,313.66,1.235230,0
1,AVGO,1380.03,1.191866,0
2,AMAT,209.82,0.806061,0
3,CMA,54.00,0.766446,1
4,CAT,358.18,0.733737,0
...,...,...,...,...
95,AAP,75.64,-0.377378,0
96,ABMD,381.02,NaN,0
97,ALXN,182.50,NaN,0
98,ATVI,94.42,NaN,0


In [10]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,ANET,313.66,1.235230,0
1,AVGO,1380.03,1.191866,0
2,AMAT,209.82,0.806061,0
3,CMA,54.00,0.766446,1
4,CAT,358.18,0.733737,0
...,...,...,...,...
95,AAP,75.64,-0.377378,0
96,ABMD,381.02,NaN,0
97,ALXN,182.50,NaN,0
98,ATVI,94.42,NaN,0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [13]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]
hqm_dataframe = pd.DataFrame(columns = hqm_columns)
for symbol in stocks['Ticker'][:100]:
    # print(symbol)
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,ADVANCED_STATS/{symbol}?token={IEX_CLOUD_API_TOKEN}'
    outer_data = requests.get(batch_api_call_url).json()
    dataOne = outer_data[0]
    dataTwo = outer_data[1]
    new_row = pd.Series([symbol, dataOne['latestPrice'], 'N/A', dataTwo['year1ChangePercent'], 'N/A', dataTwo['month6ChangePercent'],'N/A',dataTwo['month3ChangePercent'], 'N/A',dataTwo['month1ChangePercent'],'N/A','N/A'], index = hqm_columns)
    hqm_dataframe = pd.concat([hqm_dataframe, pd.DataFrame([new_row])], ignore_index=True)

C:\Users\Aryan\AppData\Local\Temp\ipykernel_27812\142848402.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqm_dataframe = pd.concat([hqm_dataframe, pd.DataFrame([new_row])], ignore_index=True)


In [22]:
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [ ]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        s = f'{change_col} ------ {percentile_col}'
        print(s)


In [38]:
from scipy import stats
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

#         change_col = f'{time_period} Price Return'
#         percentile_col = f'{time_period} Return Percentile'
#         hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row,change_col])

hqm_dataframe = hqm_dataframe.fillna(0)
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0     0.70
1     0.40
2     0.05
3     0.50
4     0.65
5     0.85
6     0.35
7     0.30
8     0.60
9     0.95
10    0.75
11    0.10
12    0.80
13    0.55
14    0.15
15    0.45
16    0.20
17    0.90
18    1.00
19    0.25
Name: One-Year Return Percentile, dtype: float64
0     1.00
1     0.90
2     0.95
3     0.30
4     0.60
5     0.55
6     0.25
7     0.50
8     0.15
9     0.05
10    0.80
11    0.10
12    0.45
13    0.35
14    0.20
15    0.65
16    0.70
17    0.40
18    0.85
19    0.75
Name: Six-Month Return Percentile, dtype: float64
0     0.75
1     0.45
2     0.85
3     0.40
4     0.20
5     0.50
6     0.35
7     0.25
8     0.15
9     0.05
10    0.70
11    0.95
12    0.30
13    0.10
14    0.65
15    0.90
16    1.00
17    0.60
18    0.80
19    0.55
Name: Three-Month Return Percentile, dtype: float64
0     0.60
1     0.95
2     0.70
3     0.65
4     0.25
5     0.15
6     0.30
7     0.10
8     0.20
9     0.35
10    0.85
11    0.50
12    0.40
13    0.05
14    0.55
15    0.90
16    1.00
17

C:\Users\Aryan\AppData\Local\Temp\ipykernel_27812\1570913552.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hqm_dataframe = hqm_dataframe.fillna(0)
C:\Users\Aryan\AppData\Local\Temp\ipykernel_27812\1570913552.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
C:\Users\Aryan\AppData\Local\Temp\ipykernel_27812\1570913552.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will r

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,151.2800,N/A,0.167409,0.70,0.379586,1.00,0.140568,0.75,0.049954,0.60,0
1,AAL,15.1900,N/A,0.064907,0.40,0.267402,0.90,0.021903,0.45,0.135361,0.95,0
2,AAP,75.6400,N/A,-0.377378,0.05,0.355975,0.95,0.172470,0.85,0.068085,0.70,0
3,AAPL,187.4300,N/A,0.090940,0.50,0.013325,0.30,0.009433,0.40,0.057975,0.65,0
4,ABBV,161.5900,N/A,0.140514,0.65,0.185383,0.60,-0.060614,0.20,-0.006162,0.25,0
5,ABC,179.9800,N/A,0.238434,0.85,0.155393,0.55,0.071500,0.50,-0.032744,0.15,0
6,ABMD,381.0200,N/A,0.000000,0.35,0.000000,0.25,0.000000,0.35,0.000000,0.30,0
7,ABT,104.0100,N/A,-0.032545,0.30,0.104481,0.50,-0.054341,0.25,-0.039776,0.10,0
8,ACN,306.9500,N/A,0.126089,0.60,-0.018702,0.15,-0.149416,0.15,-0.025333,0.20,0
9,ADBE,475.9500,N/A,0.418840,0.95,-0.212434,0.05,-0.212863,0.05,0.003923,0.35,0


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [39]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

C:\Users\Aryan\AppData\Local\Temp\ipykernel_27812\3050593765.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7625' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)


## Selecting the 20 Best Momentum Stocks

As before, we can identify the 20 best momentum stocks by sorting the DataFrame on the `HQM Score` column and dropping all but the top 20 entries.

In [40]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:20]

In [41]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,151.2800,N/A,0.167409,0.70,0.379586,1.00,0.140568,0.75,0.049954,0.60,0.7625
1,AAL,15.1900,N/A,0.064907,0.40,0.267402,0.90,0.021903,0.45,0.135361,0.95,0.6750
2,AAP,75.6400,N/A,-0.377378,0.05,0.355975,0.95,0.172470,0.85,0.068085,0.70,0.6375
3,AAPL,187.4300,N/A,0.090940,0.50,0.013325,0.30,0.009433,0.40,0.057975,0.65,0.4625
4,ABBV,161.5900,N/A,0.140514,0.65,0.185383,0.60,-0.060614,0.20,-0.006162,0.25,0.4250
5,ABC,179.9800,N/A,0.238434,0.85,0.155393,0.55,0.071500,0.50,-0.032744,0.15,0.5125
6,ABMD,381.0200,N/A,0.000000,0.35,0.000000,0.25,0.000000,0.35,0.000000,0.30,0.3125
7,ABT,104.0100,N/A,-0.032545,0.30,0.104481,0.50,-0.054341,0.25,-0.039776,0.10,0.2875
8,ACN,306.9500,N/A,0.126089,0.60,-0.018702,0.15,-0.149416,0.15,-0.025333,0.20,0.2750
9,ADBE,475.9500,N/A,0.418840,0.95,-0.212434,0.05,-0.212863,0.05,0.003923,0.35,0.3500


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [42]:
portfolio_input()

In [43]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,151.2800,2,0.167409,0.70,0.379586,1.00,0.140568,0.75,0.049954,0.60,0.7625
1,AAL,15.1900,22,0.064907,0.40,0.267402,0.90,0.021903,0.45,0.135361,0.95,0.6750
2,AAP,75.6400,4,-0.377378,0.05,0.355975,0.95,0.172470,0.85,0.068085,0.70,0.6375
3,AAPL,187.4300,1,0.090940,0.50,0.013325,0.30,0.009433,0.40,0.057975,0.65,0.4625
4,ABBV,161.5900,2,0.140514,0.65,0.185383,0.60,-0.060614,0.20,-0.006162,0.25,0.4250
5,ABC,179.9800,1,0.238434,0.85,0.155393,0.55,0.071500,0.50,-0.032744,0.15,0.5125
6,ABMD,381.0200,0,0.000000,0.35,0.000000,0.25,0.000000,0.35,0.000000,0.30,0.3125
7,ABT,104.0100,3,-0.032545,0.30,0.104481,0.50,-0.054341,0.25,-0.039776,0.10,0.2875
8,ACN,306.9500,1,0.126089,0.60,-0.018702,0.15,-0.149416,0.15,-0.025333,0.20,0.2750
9,ADBE,475.9500,0,0.418840,0.95,-0.212434,0.05,-0.212863,0.05,0.003923,0.35,0.3500
